In [ ]:
import pandas as pd
import numpy as np
import os

def limpar_id(series):
    """
    Padroniza o ID do município: remove decimais (.0), 
    converte para inteiro e preenche com zeros à esquerda (7 dígitos).
    """
    return pd.to_numeric(series, errors='coerce').fillna(0).astype(int).astype(str).str.zfill(7)

def carregar_e_preparar_dados():
    """Carrega as bases brutas e processadas aplicando a limpeza de IDs."""
    # 1. Base do Pix (Cross-Section)
    df_pix = pd.read_parquet('../data/processed/intensidade_pix_municipios.parquet')
    df_pix['id_municipio'] = limpar_id(df_pix['id_municipio'])

    # 2. Covariáveis (Socioeconômicas e Geo)
    df_covar = pd.read_parquet('../data/raw/covariaveis_municipais.parquet')
    df_covar['id_municipio'] = limpar_id(df_covar['id_municipio'])

    # 3. Homicídios (Ipeadata)
    df_homic = pd.read_csv('../data/processed/homicidios_ipeadata.csv', skiprows=1, sep=',')
    df_homic = df_homic.rename(columns={'2019': 'taxa_homicidio', 'Código': 'cod_raw'})
    df_homic['id_municipio'] = limpar_id(df_homic['cod_raw'])
    
    return df_pix, df_covar, df_homic

def realizar_merges(df_pix, df_covar, df_homic):
    """Executa a união das bases e cria a coluna de macrorregião."""
    # Pix + Covariáveis
    cols_drop = ['nome', 'sigla_uf', 'populacao', 'Municipio', 'Estado']
    df_final = df_pix.merge(
        df_covar.drop(columns=cols_drop, errors='ignore'), 
        on='id_municipio', 
        how='left'
    )

    # Adição dos Homicídios
    df_final = df_final.merge(
        df_homic[['id_municipio', 'taxa_homicidio']], 
        on='id_municipio', 
        how='left'
    )

    # CRIAR COLUNA DE MACRORREGIÃO (Dígito 1 do ID IBGE)
    # 1: Norte, 2: Nordeste, 3: Sudeste, 4: Sul, 5: Centro-Oeste
    df_final['cod_regiao'] = df_final['id_municipio'].str[0]
    
    return df_final

def tratar_e_salvar(df_final, output_path):
    """Trata valores ausentes e salva o arquivo final em parquet."""
    cols_num = [
        'taxa_homicidio', 'pib_per_capita', 'idhm_e', 
        'densidade_tel', 'populacao', 'longitude', 'latitude'
    ]

    for col in cols_num:
        if col in df_final.columns:
            df_final[col] = pd.to_numeric(df_final[col], errors='coerce').fillna(0)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df_final.to_parquet(output_path, index=False)
    
    print(f"✅ Merge concluído com sucesso!")
    print(f"📊 Coluna 'cod_regiao' criada para Exact Match.")
    print(f"📌 Arquivo salvo em: {output_path}")

def main():
    output_path = '../data/processed/dataset_final_matching.parquet'
    df_pix, df_covar, df_homic = carregar_e_preparar_dados()
    df_final = realizar_merges(df_pix, df_covar, df_homic)
    tratar_e_salvar(df_final, output_path)

if __name__ == "__main__":
    main()

✅ Merge concluído com sucesso!
📊 Municípios finais: 5570
📌 Chave id_municipio padronizada como String (7 dígitos).


🚀 Iniciando processamento...
📊 Gerando grupos de tratamento (Tercis, Quartis e Quintis)...
✅ Processo concluído!
📂 Salvo em: ../data/processed/intensidade_pix_municipios.parquet
📈 Municípios processados: 5570
🔝 Máximo Intensidade: 1.0


In [ ]:
import pandas as pd
import numpy as np
import os
import gc

# --- CONFIGURAÇÃO DE CAMINHOS ---
INPUT_FILE = '../data/raw/test.parquet'
OUTPUT_DIR = '../data/processed'
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'rais_painel_balanceado.parquet')

def processar_rais_otimizado(path):
    """
    Lê apenas colunas necessárias e usa vetorização para economizar RAM.
    """
    print("   [1/3] Lendo colunas selecionadas e mapeando setores...")
    
    # Lê apenas o necessário
    cols = ['id_municipio', 'cnae_2', 'ano', 'quantidade_vinculos_ativos']
    df = pd.read_parquet(path, columns=cols)

    # Vetorização em vez de apply: muito mais rápido e leve
    # CNAE Divisão (2 dígitos)
    div = (df['cnae_2'].fillna(0).astype(np.int32) // 1000)
    
    df['setor'] = 'Outros'
    df.loc[div.between(1, 3), 'setor'] = 'Agro'
    df.loc[div.between(5, 33), 'setor'] = 'Industria'
    df.loc[div == 84, 'setor'] = 'Setor Publico'
    df.loc[div.between(35, 99), 'setor'] = 'Servicos'
    
    # Remove coluna temporária e libera memória
    df.drop(columns=['cnae_2'], inplace=True)

    print("   [2/3] Agregando estoque...")
    df_agg = (df.groupby(['id_municipio', 'setor', 'ano'], as_index=False)
                ['quantidade_vinculos_ativos'].sum())
    
    del df
    gc.collect()
    return df_agg

def rebalancear_painel_otimizado(df_agg):
    """
    Usa tipos de dados categóricos para reduzir o tamanho do produto cartesiano.
    """
    print("   [3/3] Rebalanceando o painel...")

    # Remover 'Outros' antes de balancear economiza memória
    df_agg = df_agg[df_agg['setor'] != 'Outros'].copy()

    todos_municipios = df_agg['id_municipio'].unique()
    todos_setores = ['Agro', 'Industria', 'Servicos', 'Setor Publico']
    todos_anos = df_agg['ano'].unique()

    # Criar index e esqueleto
    index = pd.MultiIndex.from_product(
        [todos_municipios, todos_setores, todos_anos], 
        names=['id_municipio', 'setor', 'ano']
    )
    
    # Reindex é mais eficiente que Merge para balanceamento
    df_final = (df_agg.set_index(['id_municipio', 'setor', 'ano'])
                      .reindex(index, fill_value=0)
                      .reset_index())

    # Variável para DID
    df_final['log_estoque'] = np.log1p(df_final['quantidade_vinculos_ativos'].astype(np.float32))
    
    return df_final

def main():
    print("--- INICIANDO ETL RAIS OTIMIZADO ---")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    if not os.path.exists(INPUT_FILE):
        print(f"ERRO: Arquivo de entrada não encontrado: {INPUT_FILE}")
        return

    # Processamento em etapas com limpeza de cache
    df_agg = processar_rais_otimizado(INPUT_FILE)
    df_final = rebalancear_painel_otimizado(df_agg)
    
    print(f"Salvando em: {OUTPUT_FILE}")
    # compressão snappy é padrão e rápida
    df_final.to_parquet(OUTPUT_FILE, index=False, compression='snappy')
    print("--- CONCLUÍDO ---")

if __name__ == "__main__":
    main()

In [1]:
import pandas as pd

def get_simples_base(path, dtypes):
    df = pd.read_csv(path, sep=";", encoding="latin-1", header=None, 
                     usecols=[0, 4, 5, 6], names=["cnpj_basico", "mei", "ini", "fim"],
                     dtype=dtypes)
    df = df[df["mei"] == "S"].copy()
    df['ano_ini'] = (df['ini'] // 10000).fillna(0).astype("int16")
    df['ano_fim'] = (df['fim'] // 10000).fillna(0).astype("int16")
    return df.drop(columns=["mei", "ini", "fim"])

def classify_sector(df):
    divisao = (df['cnae'] // 100000).fillna(0).astype(int)
    df['setor'] = 'Servicos'
    df.loc[divisao.between(1, 3), 'setor'] = 'Agro'
    df.loc[divisao.between(5, 33), 'setor'] = 'Industria'
    df.loc[divisao == 84, 'setor'] = 'Setor Publico'
    return df

In [ ]:
import requests
import pandas as pd
from pathlib import Path

def fetch_pix_data(data_base: str):
    """Obtém dados da API do BCB para um mês específico (YYYYMM)."""
    url = f"https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/TransacoesPixPorMunicipio(DataBase=@DataBase)?@DataBase='{data_base}'&$format=json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json().get('value', [])
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return []

def save_to_parquet(data, output_path):
    """Converte para DataFrame e salva em Parquet."""
    if not data:
        return
    df = pd.DataFrame(data)
    out = Path(output_path)
    out.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(out, index=False)
    print(f"Salvo: {out} ({len(df)} registros)")

if __name__ == "__main__":
    # Exemplo de uso
    mes = '202011'
    raw_data = fetch_pix_data(mes)
    save_to_parquet(raw_data, '../data/raw/dados_pix.parquet')

Downloading: 100%|██████████|


In [4]:
def fetch_pix_data(data_base: str):
    """Obtém dados da API do BCB para um mês específico (YYYYMM)."""
    url = f"https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/TransacoesPixPorMunicipio(DataBase=@DataBase)?@DataBase='{data_base}'&$format=json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json().get('value', [])
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return []
fetch_pix_data('202011')

NameError: name 'requests' is not defined

In [ ]:
# 1. Cria a coluna unificada pegando o valor que existir
df['escolaridade'] = df['grau_instrucao_1985_2005'].fillna(df['grau_instrucao_apos_2005'])

# 2. Remove as colunas antigas para limpar o df
df.drop(columns=['grau_instrucao_1985_2005', 'grau_instrucao_apos_2005'], inplace=True)

# 3. Agrupa novamente para consolidar as linhas (necessário se os IDs eram diferentes)
df = df.groupby(['ano', 'sigla_uf', 'id_municipio', 'municipio_nome', 'escolaridade'], as_index=False)['total_vinculos'].sum()

In [2]:
import basedosdados as bd

billing_id = 'dissertacao-pnate'

query = """
  SELECT
    dados.ano as ano,
    dados.sigla_uf AS sigla_uf,
    diretorio_sigla_uf.nome AS sigla_uf_nome,
    dados.id_municipio AS id_municipio,
    diretorio_id_municipio.nome AS id_municipio_nome,
    dados.populacao as populacao
FROM `basedosdados.br_ibge_populacao.municipio` AS dados
LEFT JOIN (SELECT DISTINCT sigla,nome  FROM `basedosdados.br_bd_diretorios_brasil.uf`) AS diretorio_sigla_uf
    ON dados.sigla_uf = diretorio_sigla_uf.sigla
LEFT JOIN (SELECT DISTINCT id_municipio,nome  FROM `basedosdados.br_bd_diretorios_brasil.municipio`) AS diretorio_id_municipio
    ON dados.id_municipio = diretorio_id_municipio.id_municipio
"""

df = bd.read_sql(query = query, billing_project_id = billing_id)

Downloading: 100%|██████████|
